In [1]:
import pandas as pd
import numpy as np
import ast 
import json
import matplotlib.pyplot as plt

In [2]:
data_file = '../../data/'

## Credits DataFrame

In [3]:
df_credits = pd.read_csv(data_file + 'credits.csv')
df_credits.dtypes

cast    object
crew    object
id       int64
dtype: object

In [4]:
df_credits.iloc[0]

cast    [{'cast_id': 14, 'character': 'Woody (voice)',...
crew    [{'credit_id': '52fe4284c3a36847f8024f49', 'de...
id                                                    862
Name: 0, dtype: object

### Tao bang df_credits_clean

{ 
    id: xxx, 
    cast_1: {id: xxx, name: xxx},
    cast_2: {id: xxx, name: xxx},
    cast_3: {id: xxx, name: xxx},
    director: {id: xxx, name: xxx},
}

In [5]:
#chi lay 3 dien vien noi tieng nhat
def getMainCharacter(casts, order):
    # chuyen tu dang string sang dang list of dictionaries
    casts = ast.literal_eval(casts)
    main_char = {}
    for cast in casts:
        if cast['order'] == order:
            main_char = {'char_id':cast['id'], 'char_name':cast['name']}
    # neu khong tim thay gia tri do thi tra ve null
    if main_char:
        return main_char
    return None

In [6]:
# lay 5 dien vien chinh
def getCharacter(casts):
    casts = ast.literal_eval(casts)
    list_char = []
    for cast in casts:
        if cast['order'] < 5:
            list_char.append({ 'char_id': cast['id'], 'char_name':cast['name'] })
    # neu khong tim thay gia tri do thi tra ve null
    if list_char:
        return list_char
    return None

In [7]:
#lay dao dien dau tien trong list
def getMainDirector(crews):
    # chuyen tu dang string sang dang list of dictionaries
    crews = ast.literal_eval(crews)
    list_directors = []
    for crew in crews:
        if crew['job'] == 'Director':
            list_directors.append({ 'director_id': crew['id'], 'director_name': crew['name']})
    #neu khong co phan tu nao trong list
    if list_directors:
        return list_directors[0]
    return None

In [8]:
# lay tat ca dao dien
def getDirector(crews):
    # chuyen tu dang string sang dang list of dictionaries
    crews = ast.literal_eval(crews)
    list_directors = []
    for crew in crews:
        if crew['job'] == 'Director':
            list_directors.append({ 'director_id':crew['id'], 'director_name': crew['name']})
#     neu khong co phan tu nao trong list
    if list_directors:
        return list_directors[0:5]
    return None

In [9]:
df_credits['Main_char_0'] = df_credits['cast'].apply(getMainCharacter, args=(0,))
df_credits['Main_char_1'] = df_credits['cast'].apply(getMainCharacter, args=(1,))
df_credits['Main_char_2'] = df_credits['cast'].apply(getMainCharacter, args=(2,))
df_credits['Director'] = df_credits['crew'].apply(getMainDirector)
df_credits['Characters'] = df_credits['cast'].apply(getCharacter)
df_credits['Directors'] = df_credits['crew'].apply(getDirector)

#### Create DataFrame Credits for Model Training

In [10]:
df_credits_model = df_credits[['id', 'Main_char_0', 'Main_char_1', 'Main_char_2', 'Director'] ]

#### Create DataFrame for ES DB

In [11]:
df_credits_db = df_credits[ ['id', 'Characters', 'Directors' ] ]

## Links DataFrame 

In [12]:
df_links = pd.read_csv(data_file + 'links.csv')
df_links.dtypes

movieId      int64
imdbId       int64
tmdbId     float64
dtype: object

In [13]:
df_links.loc[1]

movieId         2.0
imdbId     113497.0
tmdbId       8844.0
Name: 1, dtype: float64

## Metadata DataFrame

In [14]:
df_metadata = pd.read_csv(data_file + 'movies_metadata.csv', low_memory=False)
df_metadata.dtypes

adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object

In [15]:
df_metadata.iloc[0]

adult                                                                False
belongs_to_collection    {'id': 10194, 'name': 'Toy Story Collection', ...
budget                                                            30000000
genres                   [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
homepage                              http://toystory.disney.com/toy-story
id                                                                     862
imdb_id                                                          tt0114709
original_language                                                       en
original_title                                                   Toy Story
overview                 Led by Woody, Andy's toys live happily in his ...
popularity                                                       21.946943
poster_path                               /rhIRbceoE9lR4veEXuwCC2wARtG.jpg
production_companies        [{'name': 'Pixar Animation Studios', 'id': 3}]
production_countries     

In [16]:
df_metadata['genres'].iloc[0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

#### Chuan hoa truong genres 

In [17]:
df_metadata['genres'].iloc[0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [18]:
def normalizeGenres(genres):
    list_genre =[]
    # by pass malformed node or string: nan error
    try:
        genres = ast.literal_eval(genres)
        for genre in genres:
            list_genre.append({'keyword_id': genre['id'], 'keyword_name': genre['name'] })
    except:
        pass
    return list_genre

In [19]:
df_metadata['genres'] = df_metadata['genres'].apply(normalizeGenres)

In [20]:
df_metadata['genres'].iloc[0]

[{'keyword_id': 16, 'keyword_name': 'Animation'},
 {'keyword_id': 35, 'keyword_name': 'Comedy'},
 {'keyword_id': 10751, 'keyword_name': 'Family'}]

#### Chuan hoa truong adult

In [21]:
def normalizeAdult(x):
    if x == 'True':
        return 1
    return 0

df_metadata['adult'] = df_metadata['adult'].apply(normalizeAdult)

#### Chuan hoa truong id

In [22]:
#list nhung id khong phai int
list_ids = df_metadata['id'].tolist()
list_not_id = []
for id in list_ids:
    if not str(id).isdigit():
        list_not_id.append(id)
print(list_not_id)

['1997-08-20', '2012-09-29', '2014-01-01']


In [23]:
# df_metadata['id'].astype(str).astype(int)
df_metadata = df_metadata[(df_metadata['id'] != '1997-08-20') & (df_metadata['id'] != '2014-01-01') & (df_metadata['id'] != '2012-09-29')  ] 

#### Chuan hoa truong vote_average 

In [24]:
df_metadata['vote_average'] = df_metadata['vote_average'].fillna(0)

#### Chuan hoa truong vote_count

In [25]:
df_metadata['vote_count'] = df_metadata['vote_count'].fillna(0)

#### Chuan hoa truong original language

In [26]:
list_lang = df_metadata['original_language'].unique().tolist()

In [27]:
def normalizeLanguage(x):
    if x == '104.0' or x == '68.0' or x == '82.0' or x is None:
        return ''
    return x

In [28]:
df_metadata['original_language'] = df_metadata['original_language'].fillna('').apply(normalizeLanguage).apply(lambda x: x.upper())

#### Chuan hoa truong spoken language

In [29]:
def normalizeSpokenLang(spoken_langs):
    #convert string of list[dict] -> list[dict]
    list_lang =[]
    # by pass malformed node or string: nan error
    try:
        spoken_langs = ast.literal_eval(spoken_langs)
        for lang in spoken_langs:
            list_lang.append({'spoken_language': lang['iso_639_1'].upper()})
    except:
        pass
    return list_lang

In [30]:
df_metadata['spoken_languages'] = df_metadata['spoken_languages'].apply(normalizeSpokenLang)

In [31]:
df_metadata['spoken_languages']

0                              [{'spoken_language': 'EN'}]
1        [{'spoken_language': 'EN'}, {'spoken_language'...
2                              [{'spoken_language': 'EN'}]
3                              [{'spoken_language': 'EN'}]
4                              [{'spoken_language': 'EN'}]
5        [{'spoken_language': 'EN'}, {'spoken_language'...
6        [{'spoken_language': 'FR'}, {'spoken_language'...
7        [{'spoken_language': 'EN'}, {'spoken_language'...
8                              [{'spoken_language': 'EN'}]
9        [{'spoken_language': 'EN'}, {'spoken_language'...
10                             [{'spoken_language': 'EN'}]
11       [{'spoken_language': 'EN'}, {'spoken_language'...
12                             [{'spoken_language': 'EN'}]
13                             [{'spoken_language': 'EN'}]
14       [{'spoken_language': 'EN'}, {'spoken_language'...
15                             [{'spoken_language': 'EN'}]
16                             [{'spoken_language': 'EN'

#### Nhung truong co budget = 0 -> revenue = 0

#### Chuan hoa truong revenue

In [32]:
min_revenue = df_metadata['revenue'].min()
max_revenue = df_metadata['revenue'].max()
print("Min revenue: {}\nMax revenue: {}".format(min_revenue, max_revenue))

Min revenue: 0.0
Max revenue: 2787965087.0


In [33]:
# thay cac cot na = 0
df_metadata['revenue'] = df_metadata['revenue'].fillna(0)

In [34]:
list_revenue = df_metadata['revenue'].tolist()

In [35]:
# plot occurence of list_revenue 
print(list_revenue)

[373554033.0, 262797249.0, 0.0, 81452156.0, 76578911.0, 187436818.0, 0.0, 0.0, 64350171.0, 352194034.0, 107879496.0, 0.0, 11348324.0, 13681765.0, 10017322.0, 116112375.0, 135000000.0, 4300000.0, 212385533.0, 35431113.0, 115101622.0, 0.0, 30303072.0, 0.0, 49800000.0, 0.0, 27400000.0, 0.0, 1738611.0, 0.0, 180000000.0, 168840000.0, 0.0, 254134910.0, 0.0, 39363635.0, 0.0, 0.0, 0.0, 676525.0, 0.0, 0.0, 0.0, 122195920.0, 21284514.0, 23574130.0, 327311859.0, 346079773.0, 0.0, 23341568.0, 0.0, 6700000.0, 0.0, 0.0, 0.0, 0.0, 17519169.0, 0.0, 0.0, 0.0, 0.0, 106269971.0, 0.0, 0.0, 0.0, 2409225.0, 0.0, 0.0, 28215918.0, 25836616.0, 11534477.0, 718490.0, 0.0, 19030691.0, 2042530.0, 5781885.0, 0.0, 0.0, 0.0, 0.0, 529766.0, 0.0, 0.0, 0.0, 0.0, 10300000.0, 0.0, 32.0, 8175346.0, 0.0, 12379402.0, 19800000.0, 0.0, 150270147.0, 0.0, 0.0, 0.0, 0.0, 0.0, 560069.0, 0.0, 0.0, 41205099.0, 182016617.0, 0.0, 34327391.0, 0.0, 0.0, 210000000.0, 28262574.0, 32392047.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 6278139.0, 0.0, 0

#### Chuan hoa truong budget

In [36]:
min_budget = df_metadata['budget'].min()
max_budget = df_metadata['budget'].max()
print("Min budget: {}\nMax budget: {}".format(min_budget, max_budget))

Min budget: 0
Max budget: 998000


In [37]:
list_budget = df_metadata['budget'].tolist()

In [38]:
df_metadata['budget'] = df_metadata['budget'].fillna(0)

#### Chuan hoa truong popularity 

In [39]:
df_metadata['popularity'] = df_metadata['popularity'].fillna(0)

#### Chuan hoa truong runtime

In [40]:
# replace missing runtime with mean value

df_metadata['runtime'] = df_metadata['runtime'].fillna((df_metadata['runtime'].mean()))

#### Chuan hoa truong belongs_to_collection

In [41]:
df_metadata['belongs_to_collection'].iloc[0]

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

In [42]:
def normalizeBelong_to_collection(belong_to_collection):
    dict_belong = {}
    try:
        belong_to_collection = ast.literal_eval(belong_to_collection)

        if belong_to_collection:
            return {"collection_id": belong_to_collection['id'], "collection_name": belong_to_collection['name'], "poster_path": belong_to_collection['poster_path']}
    except:
        pass
    return dict_belong

In [43]:
df_metadata['belongs_to_collection'] = df_metadata['belongs_to_collection'].apply(normalizeBelong_to_collection)

In [44]:
df_metadata['belongs_to_collection'].iloc[1000]

{}

#### Chuan hoa release_date

In [45]:
df_metadata['release_date'] = df_metadata['release_date'].fillna('0000-00-00')

#### Create DataFrame for ES DB

In [46]:
df_metadata_db = df_metadata[['id', 'original_title', 'imdb_id', 'original_language','adult', 'belongs_to_collection', 'budget', 'genres', 'popularity', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'poster_path','vote_average', 'vote_count' ]]

In [47]:
df_metadata_db['id'] = df_metadata_db['id'].astype('str').astype('int')

/home/vuviethung/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [48]:
df_metadata_db.dtypes

id                         int64
original_title            object
imdb_id                   object
original_language         object
adult                      int64
belongs_to_collection     object
budget                    object
genres                    object
popularity                object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
poster_path               object
vote_average             float64
vote_count               float64
dtype: object

In [49]:
df_metadata_db.iloc[0]

id                                                                     862
original_title                                                   Toy Story
imdb_id                                                          tt0114709
original_language                                                       EN
adult                                                                    0
belongs_to_collection    {'collection_id': 10194, 'collection_name': 'T...
budget                                                            30000000
genres                   [{'keyword_id': 16, 'keyword_name': 'Animation...
popularity                                                       21.946943
release_date                                                    1995-10-30
revenue                                                        3.73554e+08
runtime                                                                 81
spoken_languages                               [{'spoken_language': 'EN'}]
poster_path              

## Keywords DataFrame

In [50]:
df_keywords = pd.read_csv(data_file + 'keywords.csv')
df_keywords.dtypes

id           int64
keywords    object
dtype: object

In [51]:
df_keywords.count()

id          46419
keywords    46419
dtype: int64

In [52]:
# normalize keywords
def normalizeKeywords(keywords):
    list_keywords = []
    try:
        keywords = ast.literal_eval(keywords)
        for keyword in keywords:
            list_keywords.append({'keyword_id': keyword['id'], 'keyword_name':keyword['name']})
    except:
        pass
    return list_keywords

df_keywords['keywords'] = df_keywords['keywords'].apply(normalizeKeywords)

In [53]:
df_keywords['keywords'].iloc[0]

[{'keyword_id': 931, 'keyword_name': 'jealousy'},
 {'keyword_id': 4290, 'keyword_name': 'toy'},
 {'keyword_id': 5202, 'keyword_name': 'boy'},
 {'keyword_id': 6054, 'keyword_name': 'friendship'},
 {'keyword_id': 9713, 'keyword_name': 'friends'},
 {'keyword_id': 9823, 'keyword_name': 'rivalry'},
 {'keyword_id': 165503, 'keyword_name': 'boy next door'},
 {'keyword_id': 170722, 'keyword_name': 'new toy'},
 {'keyword_id': 187065, 'keyword_name': 'toy comes to life'}]

In [54]:
df_keywords.loc[0]

id                                                        862
keywords    [{'keyword_id': 931, 'keyword_name': 'jealousy...
Name: 0, dtype: object

### Rating DataFrame

In [55]:
df_ratings = pd.read_csv(data_file + "ratings.csv")
df_ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [56]:
df_ratings['rating']

0           1.0
1           4.5
2           5.0
3           5.0
4           5.0
5           4.0
6           4.5
7           5.0
8           4.0
9           4.0
10          5.0
11          5.0
12          4.0
13          3.5
14          4.0
15          5.0
16          5.0
17          5.0
18          5.0
19          5.0
20          2.5
21          5.0
22          5.0
23          5.0
24          0.5
25          4.0
26          5.0
27          3.0
28          3.0
29          2.0
           ... 
26024259    3.5
26024260    5.0
26024261    4.0
26024262    3.0
26024263    3.5
26024264    4.5
26024265    4.0
26024266    5.0
26024267    4.5
26024268    5.0
26024269    5.0
26024270    4.0
26024271    4.0
26024272    5.0
26024273    2.5
26024274    4.5
26024275    4.5
26024276    3.5
26024277    3.0
26024278    4.0
26024279    4.5
26024280    4.0
26024281    5.0
26024282    3.5
26024283    4.5
26024284    5.0
26024285    5.0
26024286    4.5
26024287    4.5
26024288    2.0
Name: rating, Length: 26

## Total Dataframe

In [57]:
### Join 2 bang 
df_total = df_credits_db.merge(df_metadata_db, how='inner').merge(df_keywords, how='inner')

In [58]:
df_total.iloc[0]

id                                                                     862
Characters               [{'char_id': 31, 'char_name': 'Tom Hanks'}, {'...
Directors                [{'director_id': 7879, 'director_name': 'John ...
original_title                                                   Toy Story
imdb_id                                                          tt0114709
original_language                                                       EN
adult                                                                    0
belongs_to_collection    {'collection_id': 10194, 'collection_name': 'T...
budget                                                            30000000
genres                   [{'keyword_id': 16, 'keyword_name': 'Animation...
popularity                                                       21.946943
release_date                                                    1995-10-30
revenue                                                        3.73554e+08
runtime                  

In [59]:
df_total[df_total['id'] == 862]

,id,Characters,Directors,original_title,imdb_id,original_language,adult,belongs_to_collection,budget,genres,popularity,release_date,revenue,runtime,spoken_languages,poster_path,vote_average,vote_count,keywords
0,862,"[{'char_id': 31, 'char_name': 'Tom Hanks'}, {'...","[{'director_id': 7879, 'director_name': 'John ...",Toy Story,tt0114709,EN,0,"{'collection_id': 10194, 'collection_name': 'T...",30000000,"[{'keyword_id': 16, 'keyword_name': 'Animation...",21.946943,1995-10-30,373554033.0,81.0,[{'spoken_language': 'EN'}],/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,7.7,5415.0,"[{'keyword_id': 931, 'keyword_name': 'jealousy..."


In [60]:
df_total.count()

id                       46628
Characters               44086
Directors                45711
original_title           46628
imdb_id                  46611
original_language        46628
adult                    46628
belongs_to_collection    46628
budget                   46628
genres                   46628
popularity               46628
release_date             46628
revenue                  46628
runtime                  46628
spoken_languages         46628
poster_path              46229
vote_average             46628
vote_count               46628
keywords                 46628
dtype: int64

## Write to json file

In [62]:
def writejson(row):
    out_file = data_file +'combined_data.json'
    with open( out_file, "a") as outfile:
        json.dump({"id": row['id'], 'imdb_id': row['imdb_id'], 'original_title': row['original_title'],\
                     'Characters':row['Characters'], 'Directors': row['Directors'], 'original_language': row['original_language'],\
                     'adult': row['adult'], 'belongs_to_collection':row['belongs_to_collection'],\
                     'genres': row['genres'], 'popularity': row['popularity'], 'release_date': row['release_date'],\
                     'budget': row['budget'], 'revenue': row['revenue'], 'runtime': row['runtime'], 'spoken_languages': row['spoken_languages'],\
                     'poster_path': row['poster_path'], 'vote_average': row['vote_average'], 'vote_count': row['vote_count'],\
                     'keywords': row['keywords'] }, outfile, ensure_ascii=False)
        outfile.write('\n')
df_total.apply(writejson, axis=1)

0        None
1        None
2        None
3        None
4        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23       None
24       None
25       None
26       None
27       None
28       None
29       None
         ... 
46598    None
46599    None
46600    None
46601    None
46602    None
46603    None
46604    None
46605    None
46606    None
46607    None
46608    None
46609    None
46610    None
46611    None
46612    None
46613    None
46614    None
46615    None
46616    None
46617    None
46618    None
46619    None
46620    None
46621    None
46622    None
46623    None
46624    None
46625    None
46626    None
46627    None
Length: 46628, dtype: object